In [1]:
import sys
import numpy as np
sys.path.append('../')
from deepod.models.time_series import AnomalyTransformer as model
from load_data import load_data
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 

In [2]:
train_imu,train_audio,train_spec,val_imu,val_audio,val_spec,val_imu_normal,val_audio_normal,val_spec_normal = load_data()

train_combine = np.concatenate([np.mean(train_audio,axis=-1),train_imu],axis=1)

test_audio_all = np.concatenate([val_audio_normal,val_audio],axis=0)
test_imu_all   = np.concatenate([val_imu_normal,val_imu],axis=0)
test_combine = np.concatenate([np.mean(test_audio_all,axis=-1),test_imu_all],axis=1)

labels_all = np.concatenate([np.zeros(val_audio_normal.shape[0]),np.ones(val_audio.shape[0])])

In [3]:
clf_imu = model(epochs=50,batch_size=4,seq_len=1)
clf_imu.fit(train_imu)
clf_audio = model(epochs=50,batch_size=4,seq_len=1)
clf_audio.fit(np.mean(train_audio,axis=-1))
clf_all = model(epochs=50,batch_size=4,seq_len=1)
clf_all.fit(train_combine)

epoch  1, training loss: 0.155567, time: 0.3s
epoch  2, training loss: 0.012086, time: 0.2s
epoch  3, training loss: 0.008898, time: 0.2s
epoch  4, training loss: 0.007850, time: 0.1s
epoch  5, training loss: 0.011085, time: 0.1s
epoch  6, training loss: 0.009374, time: 0.2s
epoch  7, training loss: 0.008541, time: 0.1s
epoch  8, training loss: 0.007205, time: 0.1s
epoch  9, training loss: 0.007278, time: 0.1s
epoch 10, training loss: 0.007106, time: 0.1s
epoch 11, training loss: 0.007349, time: 0.1s
epoch 12, training loss: 0.010119, time: 0.2s
epoch 13, training loss: 0.007200, time: 0.1s
epoch 14, training loss: 0.007665, time: 0.2s
epoch 15, training loss: 0.007527, time: 0.1s
epoch 16, training loss: 0.009170, time: 0.1s
epoch 17, training loss: 0.006689, time: 0.1s
epoch 18, training loss: 0.010481, time: 0.1s
epoch 19, training loss: 0.008082, time: 0.1s
epoch 20, training loss: 0.006791, time: 0.2s
epoch 21, training loss: 0.007889, time: 0.2s
epoch 22, training loss: 0.007483,

In [ ]:
scores_audio      = clf_audio.decision_function(np.mean(test_audio_all,axis=-1))
scores_imu        = clf_imu.decision_function(test_imu_all)
scores_all        = clf_all.decision_function(test_combine)

In [ ]:
#auc, mean precision, F1, percision, recall
eval_metrics_audio = ts_metrics(labels_all, scores_audio)
eval_metrics_imu = ts_metrics(labels_all, scores_imu)
result_audio = eval_metrics_audio
result_imu = eval_metrics_imu
eval_metrics_all = ts_metrics(labels_all, scores_all)
result_all = eval_metrics_all

print(result_audio)
print(result_imu)
print(result_all)

(0.8767563823471206, 0.8565451015350103, 0.8124026877942822, 0.7834757834757835, 0.843558282208589)
(0.6611996173890099, 0.5559359836198057, 0.7171550241831326, 0.5838150289017341, 0.9294478527607362)
(0.6541493502209907, 0.5275599529838066, 0.7393891592889379, 0.6112224448897795, 0.9355828220858896)
